- 되는지 안되는지 Feasibility test
    - 경량화해야함!(정확도는 유지하고 연산을 줄이는)
    - 복잡한 모델을 간단하게
- 알고리즘 경량화
    - 파라미터의 크기를 줄임.
    
<br>

## tfhub
https://tfhub.dev/
    - tensorflow에서 모델 제공

https://modelzoo.co/

- tf.keras.applications 에 있는 건 적은 편이다



In [ ]:
!pip install -U tensorflow-hub
# 설치는 - 사용은  _

In [2]:
import matplotlib.pylab as plt
import tensorflow as tf

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
import tensorflow_hub as hub

In [4]:
dir(hub)
# KerasLayer 등 layer
# multi input, output 되는 것도 sequence안에 넣을 수 있다.

['ImageModuleInfo',
 'KerasLayer',
 'LatestModuleExporter',
 'LooseVersion',
 'Module',
 'ModuleSpec',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_run',
 'absolute_import',
 'add_signature',
 'attach_image_module_info',
 'attach_message',
 'compressed_module_resolver',
 'config',
 'create_module_spec',
 'create_module_spec_from_saved_model',
 'division',
 'estimator',
 'eval_function_for_module',
 'feature_column',
 'feature_column_v2',
 'get_expected_image_size',
 'get_num_image_channels',
 'image_embedding_column',
 'image_module_info_pb2',
 'image_util',
 'keras_layer',
 'load',
 'load_module_spec',
 'logging',
 'meta_graph_lib',
 'module',
 'module_attachment_pb2',
 'module_def_pb2',
 'module_impl',
 'module_spec',
 'module_v2',
 'native_module',
 'print_function',
 'register_module_for_export',
 'registry',
 'resolve',
 'resolver',
 'saved_model_lib',
 'saved_model_mod

In [10]:
layer = hub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/classification/4',
                      input_shape=(224,224,3))
# tensorflow version 확인
# cache 기법이 아님

In [11]:
isinstance(layer, tf.keras.layers.Layer)
# 내부적으로 keras layer로 변화

True

1. Input layer
2. KerasLayer 에서 input_shape 지정
3. model.build로 input_shape 지정

In [12]:
model = tf.keras.models.Sequential([
    layer
])

# summary 에러나면 input이 없다는 뜻

In [13]:
# summary 에러나면 input이 없다는 뜻
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 1001)              1692489   
Total params: 1,692,489
Trainable params: 0
Non-trainable params: 1,692,489
_________________________________________________________________


In [18]:
layer2 = hub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/classification/4')
model2 = tf.keras.models.Sequential([
    layer2
])

In [19]:
model2.build((None,224,224,3))
# 처음에 input 안하고 동적으로 input맞추기
# build시키면 그 값으로 기억

In [20]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   (None, 1001)              1692489   
Total params: 1,692,489
Trainable params: 0
Non-trainable params: 1,692,489
_________________________________________________________________


In [21]:
model2.build((None,222,222,3))
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   (None, 1001)              1692489   
Total params: 1,692,489
Trainable params: 0
Non-trainable params: 1,692,489
_________________________________________________________________


In [26]:
model3 = tf.keras.models.Sequential([
    tf.keras.Input((224,224,3)),
    layer2
])

In [27]:
model3.built # build했는지, input넣어도 True나옴
# built가 안되면 summary가 안나옴

True

In [28]:
model3.trainable = True
model3.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   (None, 1001)              1692489   
Total params: 1,692,489
Trainable params: 1,678,409
Non-trainable params: 14,080
_________________________________________________________________


In [29]:
import numpy as np
import PIL.Image as Image

x = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')

65536/61306 [================================] - 0s 0us/step


In [30]:
data =Image.open(x)

In [33]:
model(np.array(data.resize((224,224)))[np.newaxis])

<tf.Tensor: shape=(1, 1001), dtype=float32, numpy=
array([[-0.0786503 , -0.08302835, -1.9932742 , ..., -0.09836095,
        -0.20876181,  2.6551542 ]], dtype=float32)>

In [34]:
np.argmax(model(np.array(data.resize((224,224)))[np.newaxis]).numpy())

918

In [36]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
with open(labels_path) as f:
    print(f.read().splitlines()[918])

comic book


In [39]:
p = np.array(data.resize((224,224)))
p=p/255
p = np.expand_dims(p,0)
np.argmax(model(p).numpy())

653

In [40]:
with open(labels_path) as f:
    print(f.read().splitlines()[653])

military uniform


In [41]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)
data_root

'C:\\Users\\Gyu\\.keras\\datasets\\flower_photos'

데이터 구축하는 방법
1. 이미지 형태 directory 
2. pandas DataFrame으로 관리(우아함)
  - EDA도 용이
  - tf 연동
3. HDF5 
4. imagedb

<br>

tf.keras.preprocessing.image_dataset_from_directory -> tensor로 관리
    - Autotune 등 학습 최적화
    - map 등 사용 가능하지만 사용이 어려움, augment하기엔 어려움
    

tf.keras.preprocessing.image.ImageDataGenerator : python 형태(numpy)로 바꿈
    - generator할 때마다 형태를 바꿈(augmentation 편리)
    
tf.keras.layers.experimental.preprocessing.
    - layer로 preprocessing

In [42]:
tf.keras.preprocessing.image_dataset_from_directory(data_root)

Found 3670 files belonging to 5 classes.


<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>

In [54]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
# gen은 adaptor

In [44]:
gen.flow_from_directory(data_root)
# dataframe : pandas 연동
# python 기반 generator 최적화

Found 3670 images belonging to 5 classes.


#### tfhub
- classification : 모델 전체를 가져옴
- feature-vector : fully connected 없음

### transfer learning

In [45]:
# headless model
fx = hub.KerasLayer('https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1')

In [48]:
mo = tf.keras.models.Sequential([
    tf.keras.Input((224,224,3)),
    fx,
#     tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation='softmax'),
])

In [49]:
mo.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 1792)              17673816  
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 8965      
Total params: 17,682,781
Trainable params: 8,965
Non-trainable params: 17,673,816
_________________________________________________________________


In [50]:
mo.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

#### tensor는 학습방법 5가지

1. fit
2. fit_generator
3. train_on_batch(GAN할 때 사용)
4. Gradient Tape
5. estimator

In [55]:
image_gen = gen.flow_from_directory(data_root)
mo.fit_generator(image_gen,epochs=2)

Found 3670 images belonging to 5 classes.
Epoch 1/2
115/115 [==============================] - 372s 3s/step - loss: 1.5069 - acc: 0.3708
Epoch 2/2
115/115 [==============================] - 363s 3s/step - loss: 1.2848 - acc: 0.5918


## RNN(Recurrent Neural Network)
- 순환적 신경망(재귀X)
- **상태가 중심이 된다** -> 순차적인(시간적인 상황이 결합)
    - CNN, DNN은 시점에 상관없이 같은 결과를 내지만 RNN은 시간적인 상황에 따라 나오는 모델
- 앞에 발생한 것이 일정하게 영향을 미친다는 가정(CNN의 shared weight랑 비슷한 개념)

</br>

**State와 State Transition(상태 변화)로 표현하는데 이것을 찾는 모델**
> 연속으로 일어나는 변화를 학습을 통해 찾음 -> hidden state 찾기!

- ML은 데이터 간 독립이라는 가정
- RNN은 데이터가 독립이 아니어도 상관없음

<br>


상태와 상태변이 곱 + BIAS
- 현재시점 t 에서의 hidden state: $h_t=tanh(W_{hh} h_{t-1}+W_{xh} x_t)$
- 현재시점 t 에서의 output: $y_t=W_{hy}h_t$

같은 음식인데 맛을 다르게 느끼는 이유는 상태가 다르기 때문(입력이 다르기 때문)
    - 시점마다 상태도 달라서 output도 다르다

다른 모델에 비해 인과관계가 없음

- U : 전
- V : 후
- W : State Transition

---

## 활용 방법

1. 1to1
   - vanilla NN
2. 1 to many
   - Image Captioning 
3. many to 1
   - Sentiment Classifiction
4. many to many( synched/unsynched)
   - Video Classification on frame level 
   - Machine Translation